 # Data Processing Pipeline Overview
 
This notebook processes raw data from multiple sources to produce a final, clean, and usable dataset for model building.

The steps include:
- Loading and cleaning pincode, district, and state information
- Merging population and GDP data at the pincode and state levels
- Handling missing values and ensuring data consistency
- Generating features required for modeling
 
 The final output will be a dataset with one row per pincode, containing all relevant features for downstream machine learning tasks.

# Part 1: Pincode

## Level 1

In [1]:
import pandas as pd

# Read the pincode directory CSV
df = pd.read_csv('raw_data/pincode_directory.csv')

# Print debug info
print(f"Total rows in original file: {len(df)}")
print(f"Unique pincodes in original file: {df['pincode'].nunique()}")

# For each pincode, keep only the first occurrence, but keep all columns
df = df.drop_duplicates(subset=['pincode'], keep='first')

print(f"\nRows after keeping first occurrence per pincode: {len(df)}")
print(f"Unique pincodes after this operation: {df['pincode'].nunique()}")
print(f"Columns in the file: {list(df.columns)}")

Total rows in original file: 165631
Unique pincodes in original file: 19584

Rows after keeping first occurrence per pincode: 19584
Unique pincodes after this operation: 19584
Columns in the file: ['circlename', 'regionname', 'divisionname', 'officename', 'pincode', 'officetype', 'delivery', 'district', 'statename', 'latitude', 'longitude']


In [2]:
df = df[['pincode', 'district', 'statename', 'latitude', 'longitude']]

df

,pincode,district,statename,latitude,longitude
0,507204,KHAMMAM,TELANGANA,16.9980961,80.3087675
2,507169,KHAMMAM,TELANGANA,17.0202400,80.0717300
3,507208,KHAMMAM,TELANGANA,17.155349999999999,80.216399999999993
7,507002,KHAMMAM,TELANGANA,17.2734239,80.1830938
9,507168,KHAMMAM,TELANGANA,17.596,81.056700000000006
...,...,...,...,...,...
165522,713154,PURBA BARDHAMAN,WEST BENGAL,23.2001716,88.0833558
165564,712222,HOOGHLY,WEST BENGAL,22.7921667,88.3344722
165579,711113,HOWRAH,WEST BENGAL,22.6062000,88.2940000
165594,721303,MEDINIPUR WEST,WEST BENGAL,22.3396944,87.2127500


In [3]:
# Check for NaN values in any column
num_nan_any = df.isna().any(axis=1).sum()

print(f"Number of pincodes with NaN in any column: {num_nan_any}")

# Show some examples if any exist
if num_nan_any > 0:
    print("Rows with NaN in any column:")
    print(df[df.isna().any(axis=1)])

# Remove all rows where any column is NaN
df = df.dropna()

Number of pincodes with NaN in any column: 1351
Rows with NaN in any column:
        pincode              district       statename       latitude  \
73       507315  BHADRADRI KOTHAGUDEM       TELANGANA            NaN   
103      521126               KRISHNA  ANDHRA PRADESH            NaN   
104      521122               KRISHNA  ANDHRA PRADESH            NaN   
108      521133               KRISHNA  ANDHRA PRADESH            NaN   
109      521149               KRISHNA  ANDHRA PRADESH            NaN   
...         ...                   ...             ...            ...   
163380   431025                   NaN             NaN      19.890768   
163492   796006                AIZAWL         MIZORAM            NaN   
163560   794116                   NaN             NaN  25.7307359884   
165060   210122                 BANDA   UTTAR PRADESH            NaN   
165437   736209                   NaN             NaN      26.387458   

            longitude  
73                NaN  
103       

In [4]:
df

,pincode,district,statename,latitude,longitude
0,507204,KHAMMAM,TELANGANA,16.9980961,80.3087675
2,507169,KHAMMAM,TELANGANA,17.0202400,80.0717300
3,507208,KHAMMAM,TELANGANA,17.155349999999999,80.216399999999993
7,507002,KHAMMAM,TELANGANA,17.2734239,80.1830938
9,507168,KHAMMAM,TELANGANA,17.596,81.056700000000006
...,...,...,...,...,...
165522,713154,PURBA BARDHAMAN,WEST BENGAL,23.2001716,88.0833558
165564,712222,HOOGHLY,WEST BENGAL,22.7921667,88.3344722
165579,711113,HOWRAH,WEST BENGAL,22.6062000,88.2940000
165594,721303,MEDINIPUR WEST,WEST BENGAL,22.3396944,87.2127500


In [5]:
# Add a column 'urban' to df: 1 if pincode is in raw_data/urban.csv, else 0

# Read the urban pincodes
urban_df = pd.read_csv('raw_data/urban.csv')

# Assume the urban csv has a column named 'pincode'
urban_pincodes = set(urban_df['pincode'].astype(str))

# Ensure df['pincode'] is string for comparison
df['pincode'] = df['pincode'].astype(str)

# Add the 'urban' column: 1 if pincode in urban_pincodes, else 0
df['urban'] = df['pincode'].apply(lambda x: 1 if x in urban_pincodes else 0)

df

/var/folders/lw/mxt4vf712k5_lslhrjcqqj700000gn/T/ipykernel_81702/734739172.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pincode'] = df['pincode'].astype(str)
/var/folders/lw/mxt4vf712k5_lslhrjcqqj700000gn/T/ipykernel_81702/734739172.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['urban'] = df['pincode'].apply(lambda x: 1 if x in urban_pincodes else 0)


,pincode,district,statename,latitude,longitude,urban
0,507204,KHAMMAM,TELANGANA,16.9980961,80.3087675,0
2,507169,KHAMMAM,TELANGANA,17.0202400,80.0717300,0
3,507208,KHAMMAM,TELANGANA,17.155349999999999,80.216399999999993,0
7,507002,KHAMMAM,TELANGANA,17.2734239,80.1830938,0
9,507168,KHAMMAM,TELANGANA,17.596,81.056700000000006,0
...,...,...,...,...,...,...
165522,713154,PURBA BARDHAMAN,WEST BENGAL,23.2001716,88.0833558,0
165564,712222,HOOGHLY,WEST BENGAL,22.7921667,88.3344722,1
165579,711113,HOWRAH,WEST BENGAL,22.6062000,88.2940000,1
165594,721303,MEDINIPUR WEST,WEST BENGAL,22.3396944,87.2127500,0


In [6]:
import numpy as np

# Read airport data
airport_df = pd.read_csv('raw_data/airport.csv')

# Ensure airport_df has columns: 'lat', 'lon', 'name'
airport_coords = airport_df[['lat', 'lon']].astype(float).to_numpy()
airport_names = airport_df['name'].tolist()

# Function to compute haversine distance (in km) between two points or arrays of points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    # Convert all inputs to float64 for safety
    lat1 = np.asarray(lat1, dtype=np.float64)
    lon1 = np.asarray(lon1, dtype=np.float64)
    lat2 = np.asarray(lat2, dtype=np.float64)
    lon2 = np.asarray(lon2, dtype=np.float64)
    # Convert degrees to radians
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# For each pincode, compute distance to all airports and take the minimum and the airport name
def nearest_airport_info(row):
    lat = float(row['latitude'])
    lon = float(row['longitude'])
    dists = haversine(lat, lon, airport_coords[:,0], airport_coords[:,1])
    min_idx = np.argmin(dists)
    return pd.Series({'nearest_airport_km': dists[min_idx], 'nearest_airport_name': airport_names[min_idx]})

df[['nearest_airport_km', 'nearest_airport_name']] = df.apply(nearest_airport_info, axis=1)

# Save for next cell
df

/var/folders/lw/mxt4vf712k5_lslhrjcqqj700000gn/T/ipykernel_81702/3561302029.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['nearest_airport_km', 'nearest_airport_name']] = df.apply(nearest_airport_info, axis=1)
/var/folders/lw/mxt4vf712k5_lslhrjcqqj700000gn/T/ipykernel_81702/3561302029.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['nearest_airport_km', 'nearest_airport_name']] = df.apply(nearest_airport_info, axis=1)


,pincode,district,statename,latitude,longitude,urban,nearest_airport_km,nearest_airport_name
0,507204,KHAMMAM,TELANGANA,16.9980961,80.3087675,0,73.514759,Vijayawada Airport (VGA/VOBZ)
2,507169,KHAMMAM,TELANGANA,17.0202400,80.0717300,0,94.474522,Vijayawada Airport (VGA/VOBZ)
3,507208,KHAMMAM,TELANGANA,17.155349999999999,80.216399999999993,0,92.975247,Vijayawada Airport (VGA/VOBZ)
7,507002,KHAMMAM,TELANGANA,17.2734239,80.1830938,0,94.184775,Warangal Airport (WGC/VOWA)
9,507168,KHAMMAM,TELANGANA,17.596,81.056700000000006,0,97.279182,Rajahmundry Airport (RJA/VORY)
...,...,...,...,...,...,...,...,...
165522,713154,PURBA BARDHAMAN,WEST BENGAL,23.2001716,88.0833558,0,42.227284,Guskhara Airfield
165564,712222,HOOGHLY,WEST BENGAL,22.7921667,88.3344722,1,2.776867,Barrackpore AFS (VEPI)
165579,711113,HOWRAH,WEST BENGAL,22.6062000,88.2940000,1,11.338644,Behala Airport (VEBA)
165594,721303,MEDINIPUR WEST,WEST BENGAL,22.3396944,87.2127500,0,0.181318,Kalaikunda AFS (VEDX)


In [7]:
# Save the result to processed_data folder
df.to_csv('processed_data/pincode_lev1.csv', index=False)

## Level 2

In [8]:
import pandas as pd

geoiq_csv_path = 'processed_data/geoiq_pincode_data.csv'
geoiq_df = pd.read_csv(geoiq_csv_path)

# Find rows with any NaN value in any field
geoiq_with_nan = geoiq_df[geoiq_df.isna().any(axis=1)]

print("Rows in GeoIQ CSV with at least one NaN value:")
geoiq_with_nan

Rows in GeoIQ CSV with at least one NaN value:


,url,pincode,place_name,population,area_km2,male_population,female_population


In [9]:
# Remove rows with any NaN value in any field from the original DataFrame and save to CSV
geoiq_df = geoiq_df.dropna()
geoiq_df.to_csv('processed_data/geoiq_pincode_data.csv', index=False)
geoiq_df

,url,pincode,place_name,population,area_km2,male_population,female_population
0,https://geoiq.io/places/110001---Sansad-Marg/C...,110001,110001 - Sansad Marg,250430.0,18.74,135652.0,114778.0
1,https://geoiq.io/places/110002---Indraprastha/...,110002,110002 - Indraprastha,180479.0,15.16,95754.0,84725.0
2,https://geoiq.io/places/110003---Lodi-Road/vkn...,110003,110003 - Lodi Road,187073.0,17.29,100220.0,86853.0
3,https://geoiq.io/places/110004---Rashtrapati-B...,110004,110004 - Rashtrapati Bhawan,8846.0,1.66,4851.0,3995.0
4,https://geoiq.io/places/110005---Karol-Bagh/rS...,110005,110005 - Karol Bagh,251105.0,8.61,133452.0,117653.0
...,...,...,...,...,...,...,...
18959,https://geoiq.io/places/855113---Salmari/nup9w...,855113,855113 - Salmari,385307.0,336.60,200649.0,184658.0
18960,https://geoiq.io/places/855114---Sonali/zubD8M...,855114,855114 - Sonali,256769.0,301.98,133370.0,123399.0
18961,https://geoiq.io/places/855115---Sontha/fOiVVq...,855115,855115 - Sontha,502071.0,573.34,260168.0,241903.0
18962,https://geoiq.io/places/855116---Thakurganj/at...,855116,855116 - Thakurganj,221842.0,293.80,114131.0,107711.0


In [10]:
# Add a column for population density (population per area_km2) in geoiq_df
# Only add if both population and area_km2 are present and area_km2 > 0 to avoid division by zero
geoiq_df['population_density'] = geoiq_df.apply(
    lambda row: row['population'] / row['area_km2'] if pd.notnull(row['population']) and pd.notnull(row['area_km2']) and row['area_km2'] > 0 else None,
    axis=1
)
geoiq_df

,url,pincode,place_name,population,area_km2,male_population,female_population,population_density
0,https://geoiq.io/places/110001---Sansad-Marg/C...,110001,110001 - Sansad Marg,250430.0,18.74,135652.0,114778.0,13363.393810
1,https://geoiq.io/places/110002---Indraprastha/...,110002,110002 - Indraprastha,180479.0,15.16,95754.0,84725.0,11904.947230
2,https://geoiq.io/places/110003---Lodi-Road/vkn...,110003,110003 - Lodi Road,187073.0,17.29,100220.0,86853.0,10819.722383
3,https://geoiq.io/places/110004---Rashtrapati-B...,110004,110004 - Rashtrapati Bhawan,8846.0,1.66,4851.0,3995.0,5328.915663
4,https://geoiq.io/places/110005---Karol-Bagh/rS...,110005,110005 - Karol Bagh,251105.0,8.61,133452.0,117653.0,29164.343786
...,...,...,...,...,...,...,...,...
18959,https://geoiq.io/places/855113---Salmari/nup9w...,855113,855113 - Salmari,385307.0,336.60,200649.0,184658.0,1144.702911
18960,https://geoiq.io/places/855114---Sonali/zubD8M...,855114,855114 - Sonali,256769.0,301.98,133370.0,123399.0,850.284787
18961,https://geoiq.io/places/855115---Sontha/fOiVVq...,855115,855115 - Sontha,502071.0,573.34,260168.0,241903.0,875.695050
18962,https://geoiq.io/places/855116---Thakurganj/at...,855116,855116 - Thakurganj,221842.0,293.80,114131.0,107711.0,755.078285


In [11]:
# Merge pincode_directory with geoiq_df to get pincode, state, and population
# Use the saved CSV to ensure 'pincode' column exists
pincode_dir_df = pd.read_csv('processed_data/pincode_lev1.csv')

df_demographic = pd.merge(
    pincode_dir_df,
    geoiq_df[['pincode', 'population', 'area_km2', 'population_density', 'male_population', 'female_population']],
    on='pincode',
    how='inner'
)

df_demographic

,pincode,district,statename,latitude,longitude,urban,nearest_airport_km,nearest_airport_name,population,area_km2,population_density,male_population,female_population
0,507204,KHAMMAM,TELANGANA,16.998096,80.308768,0,73.514759,Vijayawada Airport (VGA/VOBZ),26076.0,112.65,231.478029,13034.0,13042.0
1,507169,KHAMMAM,TELANGANA,17.020240,80.071730,0,94.474522,Vijayawada Airport (VGA/VOBZ),23567.0,125.51,187.769899,11710.0,11857.0
2,507208,KHAMMAM,TELANGANA,17.155350,80.216400,0,92.975247,Vijayawada Airport (VGA/VOBZ),54915.0,206.42,266.035268,27189.0,27726.0
3,507002,KHAMMAM,TELANGANA,17.273424,80.183094,0,94.184775,Warangal Airport (WGC/VOWA),209801.0,77.87,2694.246822,104113.0,105688.0
4,507168,KHAMMAM,TELANGANA,17.596000,81.056700,0,97.279182,Rajahmundry Airport (RJA/VORY),36654.0,228.09,160.699724,18600.0,18054.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17788,713217,PASCHIM BARDHAMAN,WEST BENGAL,23.572935,87.233220,0,2.536097,Raniganj AFS,5939.0,5.93,1001.517707,3112.0,2827.0
17789,713154,PURBA BARDHAMAN,WEST BENGAL,23.200172,88.083356,0,42.227284,Guskhara Airfield,10872.0,20.58,528.279883,5514.0,5358.0
17790,712222,HOOGHLY,WEST BENGAL,22.792167,88.334472,1,2.776867,Barrackpore AFS (VEPI),25902.0,15.56,1664.652956,13401.0,12501.0
17791,711113,HOWRAH,WEST BENGAL,22.606200,88.294000,1,11.338644,Behala Airport (VEBA),25701.0,1.98,12980.303030,13364.0,12337.0


In [12]:
# Save the result to processed_data folder
df_demographic.to_csv('processed_data/pincode_lev2.csv', index=False)

## Level 3

In [13]:
import pandas as pd

# Read state-wise GDP data
state_gdp_df = pd.read_csv('raw_data/state_wise_gdp.csv')

# Rename 'gdp' column to 'state_gdp'
state_gdp_df = state_gdp_df.rename(columns={'gdp': 'state_gdp'})

# Find states with NaN state_gdp values
states_with_nan_gdp = state_gdp_df[state_gdp_df['state_gdp'].isna()]['state']
print("States with NaN GDP values:")
print(states_with_nan_gdp.tolist())
print(f"Number of states with NaN GDP: {len(states_with_nan_gdp)}")

# Remove states with NaN state_gdp values
state_gdp_df = state_gdp_df.dropna(subset=['state_gdp'])
state_gdp_df

States with NaN GDP values:
['GOA', 'CHANDIGARH', 'MANIPUR', 'NAGALAND', 'ARUNACHAL PRADESH', 'MIZORAM', 'ANDAMAN AND NICOBAR ISLANDS']
Number of states with NaN GDP: 7


,state,state_gdp
0,MAHARASHTRA,40443.0
1,TAMIL NADU,27216.0
2,UTTAR PRADESH,25479.0
3,KARNATAKA,25007.0
4,GUJARAT,24258.0
5,WEST BENGAL,17009.0
6,RAJASTHAN,15284.0
7,TELANGANA,15020.0
8,ANDHRA PRADESH,14397.0
9,MADHYA PRADESH,13633.0


In [14]:
# Read pincode-directory to get pincode and state
df = pd.read_csv('processed_data/pincode_lev2.csv')

# Rename 'population' to 'pincode_population'
df = df.rename(columns={'population': 'pincode_population'})

# Merge to get GDP for each pincode's state, and drop the duplicate 'state' column
df = pd.merge(
    df,
    state_gdp_df[['state', 'state_gdp']],
    left_on='statename',
    right_on='state',
    how='inner'
).drop(columns=['state'])

df

,pincode,district,statename,latitude,longitude,urban,nearest_airport_km,nearest_airport_name,pincode_population,area_km2,population_density,male_population,female_population,state_gdp
0,507204,KHAMMAM,TELANGANA,16.998096,80.308768,0,73.514759,Vijayawada Airport (VGA/VOBZ),26076.0,112.65,231.478029,13034.0,13042.0,15020.0
1,507169,KHAMMAM,TELANGANA,17.020240,80.071730,0,94.474522,Vijayawada Airport (VGA/VOBZ),23567.0,125.51,187.769899,11710.0,11857.0,15020.0
2,507208,KHAMMAM,TELANGANA,17.155350,80.216400,0,92.975247,Vijayawada Airport (VGA/VOBZ),54915.0,206.42,266.035268,27189.0,27726.0,15020.0
3,507002,KHAMMAM,TELANGANA,17.273424,80.183094,0,94.184775,Warangal Airport (WGC/VOWA),209801.0,77.87,2694.246822,104113.0,105688.0,15020.0
4,507168,KHAMMAM,TELANGANA,17.596000,81.056700,0,97.279182,Rajahmundry Airport (RJA/VORY),36654.0,228.09,160.699724,18600.0,18054.0,15020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17464,713217,PASCHIM BARDHAMAN,WEST BENGAL,23.572935,87.233220,0,2.536097,Raniganj AFS,5939.0,5.93,1001.517707,3112.0,2827.0,17009.0
17465,713154,PURBA BARDHAMAN,WEST BENGAL,23.200172,88.083356,0,42.227284,Guskhara Airfield,10872.0,20.58,528.279883,5514.0,5358.0,17009.0
17466,712222,HOOGHLY,WEST BENGAL,22.792167,88.334472,1,2.776867,Barrackpore AFS (VEPI),25902.0,15.56,1664.652956,13401.0,12501.0,17009.0
17467,711113,HOWRAH,WEST BENGAL,22.606200,88.294000,1,11.338644,Behala Airport (VEBA),25701.0,1.98,12980.303030,13364.0,12337.0,17009.0


In [15]:
# Add a new column to df with the total population for each pincode's state
df['state_population'] = df['statename'].map(
    df.groupby('statename')['pincode_population'].sum()
)

df

,pincode,district,statename,latitude,longitude,urban,nearest_airport_km,nearest_airport_name,pincode_population,area_km2,population_density,male_population,female_population,state_gdp,state_population
0,507204,KHAMMAM,TELANGANA,16.998096,80.308768,0,73.514759,Vijayawada Airport (VGA/VOBZ),26076.0,112.65,231.478029,13034.0,13042.0,15020.0,35287874.0
1,507169,KHAMMAM,TELANGANA,17.020240,80.071730,0,94.474522,Vijayawada Airport (VGA/VOBZ),23567.0,125.51,187.769899,11710.0,11857.0,15020.0,35287874.0
2,507208,KHAMMAM,TELANGANA,17.155350,80.216400,0,92.975247,Vijayawada Airport (VGA/VOBZ),54915.0,206.42,266.035268,27189.0,27726.0,15020.0,35287874.0
3,507002,KHAMMAM,TELANGANA,17.273424,80.183094,0,94.184775,Warangal Airport (WGC/VOWA),209801.0,77.87,2694.246822,104113.0,105688.0,15020.0,35287874.0
4,507168,KHAMMAM,TELANGANA,17.596000,81.056700,0,97.279182,Rajahmundry Airport (RJA/VORY),36654.0,228.09,160.699724,18600.0,18054.0,15020.0,35287874.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17464,713217,PASCHIM BARDHAMAN,WEST BENGAL,23.572935,87.233220,0,2.536097,Raniganj AFS,5939.0,5.93,1001.517707,3112.0,2827.0,17009.0,98680325.0
17465,713154,PURBA BARDHAMAN,WEST BENGAL,23.200172,88.083356,0,42.227284,Guskhara Airfield,10872.0,20.58,528.279883,5514.0,5358.0,17009.0,98680325.0
17466,712222,HOOGHLY,WEST BENGAL,22.792167,88.334472,1,2.776867,Barrackpore AFS (VEPI),25902.0,15.56,1664.652956,13401.0,12501.0,17009.0,98680325.0
17467,711113,HOWRAH,WEST BENGAL,22.606200,88.294000,1,11.338644,Behala Airport (VEBA),25701.0,1.98,12980.303030,13364.0,12337.0,17009.0,98680325.0


In [16]:
# Calculate pincode GDP: (pincode population / state total population) * state GDP
df['pincode_gdp'] = (
    df['pincode_population'] / df['state_population']
) * df['state_gdp']

df

,pincode,district,statename,latitude,longitude,urban,nearest_airport_km,nearest_airport_name,pincode_population,area_km2,population_density,male_population,female_population,state_gdp,state_population,pincode_gdp
0,507204,KHAMMAM,TELANGANA,16.998096,80.308768,0,73.514759,Vijayawada Airport (VGA/VOBZ),26076.0,112.65,231.478029,13034.0,13042.0,15020.0,35287874.0,11.099040
1,507169,KHAMMAM,TELANGANA,17.020240,80.071730,0,94.474522,Vijayawada Airport (VGA/VOBZ),23567.0,125.51,187.769899,11710.0,11857.0,15020.0,35287874.0,10.031104
2,507208,KHAMMAM,TELANGANA,17.155350,80.216400,0,92.975247,Vijayawada Airport (VGA/VOBZ),54915.0,206.42,266.035268,27189.0,27726.0,15020.0,35287874.0,23.374128
3,507002,KHAMMAM,TELANGANA,17.273424,80.183094,0,94.184775,Warangal Airport (WGC/VOWA),209801.0,77.87,2694.246822,104113.0,105688.0,15020.0,35287874.0,89.300110
4,507168,KHAMMAM,TELANGANA,17.596000,81.056700,0,97.279182,Rajahmundry Airport (RJA/VORY),36654.0,228.09,160.699724,18600.0,18054.0,15020.0,35287874.0,15.601481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17464,713217,PASCHIM BARDHAMAN,WEST BENGAL,23.572935,87.233220,0,2.536097,Raniganj AFS,5939.0,5.93,1001.517707,3112.0,2827.0,17009.0,98680325.0,1.023674
17465,713154,PURBA BARDHAMAN,WEST BENGAL,23.200172,88.083356,0,42.227284,Guskhara Airfield,10872.0,20.58,528.279883,5514.0,5358.0,17009.0,98680325.0,1.873949
17466,712222,HOOGHLY,WEST BENGAL,22.792167,88.334472,1,2.776867,Barrackpore AFS (VEPI),25902.0,15.56,1664.652956,13401.0,12501.0,17009.0,98680325.0,4.464589
17467,711113,HOWRAH,WEST BENGAL,22.606200,88.294000,1,11.338644,Behala Airport (VEBA),25701.0,1.98,12980.303030,13364.0,12337.0,17009.0,98680325.0,4.429944


In [17]:
# Save the pincode-wise GDP data to the processed_data folder
df.to_csv('processed_data/pincode_lev3.csv', index=False)

# Part 2: Rental

In [18]:
import pandas as pd

# Read the rental CSV from the raw_data folder
rental_df = pd.read_csv('raw_data/rental.csv')

# Print the dataframe
print("Rental DataFrame:")
rental_df

Rental DataFrame:


,code,type,sqft,rent,rent_per_sqft
0,A01,Hub,47592.0,"665,637.00",13.99
1,A01,Hub,62641.0,"1,103,550.00",17.62
2,A02,Branch,3272.0,"75,200.00",22.98
3,A03,Branch,4300.0,"136,500.00",31.74
4,A04,Hub,30376.0,"713,836.00",23.50
...,...,...,...,...,...
452,E08,Branch,5620.0,"229,336.00",40.81
453,Q34,Branch,500.0,"14,333.00",28.67
454,U27,Branch,2400.0,"30,000.00",12.50
455,Q10,Branch,150.0,"5,775.00",38.50


In [19]:
# Add a new column 'type_encode' to rental_df: 'Hub' -> 0, 'Branch' -> 1, and place it after 'type'
rental_df['type_encode'] = rental_df['type'].map({'Hub': 0, 'Branch': 1})
# Move 'type_encode' to be immediately after 'type'
cols = list(rental_df.columns)
if 'type' in cols and 'type_encode' in cols:
    type_idx = cols.index('type')
    cols.insert(type_idx + 1, cols.pop(cols.index('type_encode')))
    rental_df = rental_df[cols]

# Optionally, print the head of the updated DataFrame
print("Rental DataFrame with type_encode after type:")
rental_df

Rental DataFrame with type_encode after type:


,code,type,type_encode,sqft,rent,rent_per_sqft
0,A01,Hub,0,47592.0,"665,637.00",13.99
1,A01,Hub,0,62641.0,"1,103,550.00",17.62
2,A02,Branch,1,3272.0,"75,200.00",22.98
3,A03,Branch,1,4300.0,"136,500.00",31.74
4,A04,Hub,0,30376.0,"713,836.00",23.50
...,...,...,...,...,...,...
452,E08,Branch,1,5620.0,"229,336.00",40.81
453,Q34,Branch,1,500.0,"14,333.00",28.67
454,U27,Branch,1,2400.0,"30,000.00",12.50
455,Q10,Branch,1,150.0,"5,775.00",38.50


In [20]:
# Read the DTDC pincode mapping CSV
pincode_df = pd.read_csv('raw_data/dtdc_pincode_mapping.csv')

# Merge rental_df with pincode_df on the 'code' column to get the pincode
rental_df = rental_df.merge(pincode_df[['office_code', 'pincode']], left_on='code', right_on='office_code', how='left')

# Ensure the pincode column is of integer type (not float)
rental_df['pincode'] = rental_df['pincode'].astype('Int64')  # Use 'Int64' to allow for NA values if any

# Optionally, drop the 'office_code' column if not needed
rental_df = rental_df.drop(columns=['office_code'])

# Reorder columns to make 'pincode' the first column
cols = list(rental_df.columns)
if 'pincode' in cols:
    cols.insert(0, cols.pop(cols.index('pincode')))
    rental_df = rental_df[cols]

# Show the updated dataframe
print("Rental DataFrame with Pincode:")
rental_df

Rental DataFrame with Pincode:


,pincode,code,type,type_encode,sqft,rent,rent_per_sqft
0,382330,A01,Hub,0,47592.0,"665,637.00",13.99
1,382330,A01,Hub,0,62641.0,"1,103,550.00",17.62
2,390016,A02,Branch,1,3272.0,"75,200.00",22.98
3,360002,A03,Branch,1,4300.0,"136,500.00",31.74
4,394325,A04,Hub,0,30376.0,"713,836.00",23.50
...,...,...,...,...,...,...,...
452,625010,E08,Branch,1,5620.0,"229,336.00",40.81
453,123501,Q34,Branch,1,500.0,"14,333.00",28.67
454,244713,U27,Branch,1,2400.0,"30,000.00",12.50
455,306401,Q10,Branch,1,150.0,"5,775.00",38.50


In [21]:
# For duplicate pincodes with the same type, take the mean rent_per_sqft
# Also, print their code and type. Check if pincode/type_encode always matches code/type.

# Save the original code/type for each (pincode, type_encode) before grouping
code_type_lookup = rental_df.groupby(['pincode', 'type_encode'])[['code', 'type']].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0]).reset_index()

# Group by pincode and type_encode, take mean rent_per_sqft
rental_df_grouped = rental_df.groupby(['pincode', 'type_encode'], as_index=False)['rent_per_sqft'].mean()

# Merge back code and type for display
rental_df = pd.merge(rental_df_grouped, code_type_lookup, on=['pincode', 'type_encode'], how='left')

# Reorder columns to have code and type before type_encode
cols = list(rental_df.columns)
for col in ['code', 'type']:
    if col in cols:
        cols.insert(cols.index('type_encode'), cols.pop(cols.index(col)))
rental_df = rental_df[cols]

# Check for mismatches: for each row, does type_encode match type ('Hub'->0, 'Branch'->1), and does code match original code for that pincode/type_encode?
type_map = {'Hub': 0, 'Branch': 1}
mismatch_mask = rental_df.apply(lambda row: type_map.get(row['type'], -1) != row['type_encode'], axis=1)
num_mismatches = mismatch_mask.sum()

if num_mismatches > 0:
    print(f"\nNumber of rows where type_encode does NOT match type: {num_mismatches}")
    print(rental_df[mismatch_mask][['pincode', 'type_encode', 'type', 'code']])
else:
    print("\nAll rows have matching type_encode and type.")

rental_df


All rows have matching type_encode and type.


,pincode,code,type,type_encode,rent_per_sqft
0,110002,N33,Branch,1,83.130
1,110017,S01,Branch,1,149.410
2,110020,S13,Branch,1,40.482
3,110025,S17,Branch,1,98.290
4,110028,N03,Branch,1,72.930
...,...,...,...,...,...
376,834010,T02,Hub,0,21.060
377,842002,T05,Branch,1,29.420
378,845401,T36,Branch,1,25.890
379,846004,T26,Branch,1,32.870


In [22]:
# Find and print rows with the same pincode but different type
# This means: for each pincode, if there is more than one type (0 and 1), print those rows

# Find pincodes that have both types (0 and 1)
pincode_type_counts = rental_df.groupby('pincode')['type_encode'].nunique()
pincodes_with_both_types = pincode_type_counts[pincode_type_counts > 1].index

# Filter rental_df for these pincodes
rows_with_both_types = rental_df[rental_df['pincode'].isin(pincodes_with_both_types)]

print("Rows with same pincode but different type:")
print(rows_with_both_types)

print(f"\nNumber of such rows: {len(rows_with_both_types)}")

Rows with same pincode but different type:
     pincode  code    type  type_encode  rent_per_sqft
5     110037   N05     Hub            0         36.400
6     110037   S03  Branch            1         95.195
17    122001   L04     Hub            0         44.912
18    122001   L14  Branch            1         33.800
71    226002   U43     Hub            0         19.950
72    226002  U139  Branch            1         19.000
104   302020   Q05     Hub            0         19.720
105   302020   Q17  Branch            1         22.900
146   401208   M81     Hub            0         50.000
147   401208   M46  Branch            1         71.700
234   560064   B10     Hub            0         15.620
235   560064   B26  Branch            1         32.080
323   711409   K95     Hub            0         25.470
324   711409   K72  Branch            1         24.260
335   734010   K66     Hub            0         14.930
336   734010  K129  Branch            1         15.440

Number of such rows: 

In [23]:
rental_df.to_csv('processed_data/rental.csv', index=False)

## Part 3: Final Data

In [24]:
import pandas as pd

# Read the rental and data CSV files
rental_df = pd.read_csv('processed_data/rental.csv')
pincode_df = pd.read_csv('processed_data/pincode_lev3.csv')

# Merge: keep only rental rows, append all columns from data.csv
final_df = pd.merge(rental_df, pincode_df, on='pincode', how='left')

# Show the resulting DataFrame
final_df

,pincode,code,type,type_encode,rent_per_sqft,district,statename,latitude,longitude,urban,nearest_airport_km,nearest_airport_name,pincode_population,area_km2,population_density,male_population,female_population,state_gdp,state_population,pincode_gdp
0,110002,N33,Branch,1,83.130,CENTRAL,DELHI,28.644500,77.223139,1.0,6.883031,Safdarjung Airport (VIDD),180479.0,15.16,11904.947230,95754.0,84725.0,11077.0,18941837.0,105.542344
1,110017,S01,Branch,1,149.410,SOUTH,DELHI,28.528842,77.222557,1.0,6.401485,Safdarjung Airport (VIDD),236041.0,10.41,22674.447646,126212.0,109829.0,11077.0,18941837.0,138.034456
2,110020,S13,Branch,1,40.482,SOUTH,DELHI,28.520000,77.290000,1.0,10.911746,Safdarjung Airport (VIDD),202963.0,8.44,24047.748815,110392.0,92571.0,11077.0,18941837.0,118.690766
3,110025,S17,Branch,1,98.290,BUDAUN,UTTAR PRADESH,28.542556,77.293167,1.0,9.723582,Safdarjung Airport (VIDD),269837.0,17.03,15844.803288,145551.0,124286.0,25479.0,225257355.0,30.521431
4,110028,N03,Branch,1,72.930,NEW DELHI,DELHI,28.632141,77.138852,0.0,7.971344,New Delhi International Airport (DEL/VIDP),119198.0,4.80,24832.916667,66506.0,52692.0,11077.0,18941837.0,69.705818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,834010,T02,Hub,0,21.060,RANCHI,JHARKHAND,23.321460,85.364005,0.0,4.392521,Ranchi Airport (IXR/VERC),141654.0,450.35,314.542023,71790.0,69864.0,4610.0,36380707.0,17.949759
377,842002,T05,Branch,1,29.420,MUZAFFARPUR,BIHAR,26.120888,85.364720,1.0,5.255898,Muzzafarpur Airport (MZU/VEMZ),151334.0,21.02,7199.524263,79950.0,71384.0,8544.0,122485421.0,10.556340
378,845401,T36,Branch,1,25.890,PURBI CHAMPARAN,BIHAR,26.565600,85.137560,1.0,52.681254,Muzzafarpur Airport (MZU/VEMZ),277607.0,188.01,1476.554439,147856.0,129751.0,8544.0,122485421.0,19.364543
379,846004,T26,Branch,1,32.870,DARBHANGA,BIHAR,26.156167,85.892889,1.0,4.800151,Darbhanga AFS,47077.0,3.95,11918.227848,24806.0,22271.0,8544.0,122485421.0,3.283867


In [25]:
# Print rows that have any NaN values in any columns
nan_rows = final_df[final_df.isna().any(axis=1)]
print("Rows with NaN values in any column:")
print(nan_rows)
print(f"\nNumber of rows with NaN values: {len(nan_rows)}")

Rows with NaN values in any column:
     pincode  code    type  type_encode  rent_per_sqft district statename  \
21    122050   L24     Hub            0         17.530      NaN       NaN   
22    122107   L28  Branch            1         11.030      NaN       NaN   
23    123106   L05  Branch            1         12.410      NaN       NaN   
24    123501   Q34  Branch            1         17.905      NaN       NaN   
46    160002   J01  Branch            1         78.750      NaN       NaN   
59    201308   N35  Branch            1         26.250      NaN       NaN   
65    209726  U141  Branch            1         51.450      NaN       NaN   
130   395007   A46  Branch            1         48.000      NaN       NaN   
150   403005   B32  Branch            1         36.750      NaN       NaN   
151   403601   B51  Branch            1         39.190      NaN       NaN   
152   403722   B65  Branch            1         20.300      NaN       NaN   
157   411015   P01     Hub            0 

In [26]:
# Remove all rows with any NaN values
final_df = final_df.dropna()

final_df

,pincode,code,type,type_encode,rent_per_sqft,district,statename,latitude,longitude,urban,nearest_airport_km,nearest_airport_name,pincode_population,area_km2,population_density,male_population,female_population,state_gdp,state_population,pincode_gdp
0,110002,N33,Branch,1,83.130,CENTRAL,DELHI,28.644500,77.223139,1.0,6.883031,Safdarjung Airport (VIDD),180479.0,15.16,11904.947230,95754.0,84725.0,11077.0,18941837.0,105.542344
1,110017,S01,Branch,1,149.410,SOUTH,DELHI,28.528842,77.222557,1.0,6.401485,Safdarjung Airport (VIDD),236041.0,10.41,22674.447646,126212.0,109829.0,11077.0,18941837.0,138.034456
2,110020,S13,Branch,1,40.482,SOUTH,DELHI,28.520000,77.290000,1.0,10.911746,Safdarjung Airport (VIDD),202963.0,8.44,24047.748815,110392.0,92571.0,11077.0,18941837.0,118.690766
3,110025,S17,Branch,1,98.290,BUDAUN,UTTAR PRADESH,28.542556,77.293167,1.0,9.723582,Safdarjung Airport (VIDD),269837.0,17.03,15844.803288,145551.0,124286.0,25479.0,225257355.0,30.521431
4,110028,N03,Branch,1,72.930,NEW DELHI,DELHI,28.632141,77.138852,0.0,7.971344,New Delhi International Airport (DEL/VIDP),119198.0,4.80,24832.916667,66506.0,52692.0,11077.0,18941837.0,69.705818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,834010,T02,Hub,0,21.060,RANCHI,JHARKHAND,23.321460,85.364005,0.0,4.392521,Ranchi Airport (IXR/VERC),141654.0,450.35,314.542023,71790.0,69864.0,4610.0,36380707.0,17.949759
377,842002,T05,Branch,1,29.420,MUZAFFARPUR,BIHAR,26.120888,85.364720,1.0,5.255898,Muzzafarpur Airport (MZU/VEMZ),151334.0,21.02,7199.524263,79950.0,71384.0,8544.0,122485421.0,10.556340
378,845401,T36,Branch,1,25.890,PURBI CHAMPARAN,BIHAR,26.565600,85.137560,1.0,52.681254,Muzzafarpur Airport (MZU/VEMZ),277607.0,188.01,1476.554439,147856.0,129751.0,8544.0,122485421.0,19.364543
379,846004,T26,Branch,1,32.870,DARBHANGA,BIHAR,26.156167,85.892889,1.0,4.800151,Darbhanga AFS,47077.0,3.95,11918.227848,24806.0,22271.0,8544.0,122485421.0,3.283867


In [27]:
final_df.to_csv('data.csv', index=False)